In [1]:
import tushare as ts

pro = ts.pro_api('6858effdee9ed9ac0657625593e785aa8069b84b0de9016e5d3cdb3e') 
df = pro.query('daily', ts_code = '000001.SZ') 


In [3]:
import pandas as pd

df = pd.DataFrame(df)

df.to_csv('000001SZ.csv')

In [99]:
import numpy as np
from sklearn import preprocessing

def general_data(df, forecast_out):
    df = df[['open', 'high', 'low', 'close', 'vol']] 
    df['HL_PCT'] = (df['high'] - df['close']) / df['close'] * 100.0 
    df['PCT_change'] = (df['close'] - df['open']) / df['open'] * 100.0 
        
    df = df[['close', 'HL_PCT', 'PCT_change', 'vol']] 
       
    df.fillna(-99999, inplace=True)
    df['label'] = df['close'].shift(-forecast_out) 
    
    X = np.array(df.drop(['label'], 1)) 
    X = preprocessing.scale(X) 
        
    X_lately = X[0:forecast_out,:] 

    X = X[:-forecast_out]
        
    df.dropna(inplace=True) 
    y = np.array(df['label'])
                        
    return X, y, X_lately

In [114]:
X, y, X_lately = general_data(df, 7)

d:\python\python37\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
d:\python\python37\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [109]:
X

array([[ 0.11987231, -0.51923416, -0.41947328,  0.89619953],
       [ 0.14665364, -1.10791402,  0.94117834,  1.87339088],
       [ 0.09309097, -0.60416627,  0.80885772,  1.81980532],
       ...,
       [ 0.00487011,  0.46181549, -0.13272543, -0.70805551],
       [-0.01088362, -0.00624666,  0.02214876, -0.73686472],
       [-0.01088362, -0.24573956, -0.4411841 , -0.73003052]])

In [110]:
from sklearn import model_selection, svm
from sklearn.linear_model import LinearRegression

def general_linear(X, y, X_lately):
    
    accuracy = 0
    forecast_set = []
    
    X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.2) # 把X和y分成训练和测试两部分
    
    clf = LinearRegression(n_jobs=-1)
    clf.fit(X_train, y_train)
    
    
    accuracy = clf.score(X_test, y_test)
    forecast_set = clf.predict(X_lately)
          
    return accuracy, forecast_set

In [115]:
accuracy, forecast_set = general_linear(X, y, X_lately)

In [116]:
forecast_set

array([15.1907    , 14.79259351, 14.52862888, 14.60411149, 14.63444771,
       14.40598741, 14.29893824])